In [25]:
import os, textwrap, math, shutil, random, time
import numpy as np
import gymnasium as gym
import pybullet as p
import pybullet_data
from gymnasium import spaces

In [21]:
os.makedirs("meshes", exist_ok=True)   # put Body_Fish.stl, RearFin_Fish.stl, FrontFin_Fish.stl here

URDF_TEXT = textwrap.dedent("""
<robot name="simple_fish">

  <!-- ▸ BODY (parent link) -->
  <link name="body">
    <inertial>
      <mass value="0.15"/>
      <inertia ixx="2e-4" iyy="2e-4" izz="2e-4"/>
    </inertial>
    <visual>
      <geometry>
        <mesh filename="meshes/Body_Fish.stl" scale="0.001 0.001 0.001"/>
      </geometry>
    </visual>
    <collision>
      <geometry>
        <mesh filename="meshes/Body_Fish.stl" scale="0.001 0.001 0.001"/>
      </geometry>
    </collision>
  </link>

  <!-- ▸ TAIL (hinged) -->
  <link name="tail">
    <inertial>
      <mass value="0.05"/>
      <inertia ixx="5e-5" iyy="5e-5" izz="5e-5"/>
    </inertial>
    <visual>
      <geometry>
        <mesh filename="meshes/RearFin_Fish.stl" scale="0.001 0.001 0.001"/>
      </geometry>
    </visual>
    <collision>
      <geometry>
        <mesh filename="meshes/RearFin_Fish.stl" scale="0.001 0.001 0.001"/>
      </geometry>
    </collision>
  </link>

  <!-- ▸ FRONT FIN (fixed) -->
  <link name="front_fin">
    <inertial>
      <mass value="0.02"/>
      <inertia ixx="2e-5" iyy="2e-5" izz="2e-5"/>
    </inertial>
    <visual>
      <geometry>
        <mesh filename="meshes/FrontFin_Fish.stl" scale="0.001 0.001 0.001"/>
      </geometry>
    </visual>
    <collision>
      <geometry>
        <mesh filename="meshes/FrontFin_Fish.stl" scale="0.001 0.001 0.001"/>
      </geometry>
    </collision>
  </link>

  <!-- revolute hinge: body ↔ tail -->
  <joint name="tail_hinge" type="revolute">
    <parent link="body"/>
    <child  link="tail"/>
    <origin xyz="0 -0.15 0" rpy="0 0 0"/>
    <axis xyz="0 0 1"/>
    <limit lower="-1.0" upper="1.0" effort="1.0" velocity="10.0"/>
  </joint>

  <!-- fixed joint: body ↔ front fin -->
  <joint name="fin_pin" type="fixed">
    <parent link="body"/>
    <child  link="front_fin"/>
    <origin xyz="0.05 0 -0.02" rpy="0 0 0"/>
  </joint>

</robot>
""").strip()

with open("simple_fish.urdf", "w") as f:
    f.write(URDF_TEXT)

print("✓ Updated URDF written")

✓ Updated URDF written


In [22]:
WATER_RHO = 1000        # kg m‑3
g          = 9.81

def apply_buoyancy_and_drag(body_id, volume=2e-4, lin_drag=3.0, ang_drag=2e-3):
    """Adds crude buoyancy and isotropic linear / angular damping each sim step."""
    # Buoyancy
    p.applyExternalForce(body_id, -1, (0, 0, WATER_RHO*volume*g), (0,0,0),
                         p.LINK_FRAME)
    # Drag
    lin_vel, ang_vel = p.getBaseVelocity(body_id)
    p.applyExternalForce(body_id, -1,
        (-lin_drag*lin_vel[0], -lin_drag*lin_vel[1], -lin_drag*lin_vel[2]),
        (0,0,0), p.LINK_FRAME)
    p.applyExternalTorque(body_id, -1,
        (-ang_drag*ang_vel[0], -ang_drag*ang_vel[1], -ang_drag*ang_vel[2]),
        p.LINK_FRAME)

In [28]:
class FishEnv(gym.Env):
    """One‑servo biomimetic fish with PyBullet water hack."""
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 60}

    def __init__(self, render_mode=None, max_torque=0.3):
        super().__init__()
        self.render_mode, self.max_torque = render_mode, max_torque
        self.observation_space = spaces.Box(-np.inf, np.inf, shape=(11,))
        self.action_space      = spaces.Box(-1.0, 1.0, shape=(1,))

        if p.isConnected():                 # ensure single session
            p.disconnect()
        if render_mode == "human":
            cid = p.connect(p.SHARED_MEMORY)
            if cid < 0:
                p.connect(p.GUI)
        else:
            p.connect(p.DIRECT)

        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -g)

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        p.resetSimulation()
        p.loadURDF("plane.urdf")            # seafloor
        self.fish_id = p.loadURDF("simple_fish.urdf", [0,0,-0.5])
        self.tail_j  = 0
        p.setJointMotorControl2(self.fish_id, self.tail_j,
                                p.VELOCITY_CONTROL, force=0)   # disable default
        return self._get_obs(), {}

    def step(self, action):
        torque = float(np.clip(action[0], -1, 1) * self.max_torque)
        p.setJointMotorControl2(self.fish_id, self.tail_j, p.TORQUE_CONTROL, force=torque)

        apply_buoyancy_and_drag(self.fish_id)
        p.stepSimulation()

        obs   = self._get_obs()
        rew   = self._reward(obs, torque)
        term  = trunc = False
        if self.render_mode == "human":
            time.sleep(1/240)
        return obs, rew, term, trunc, {}

    # ---- helpers ----
    def _get_obs(self):
        pos, orn  = p.getBasePositionAndOrientation(self.fish_id)
        lin_v, ang_v = p.getBaseVelocity(self.fish_id)
        depth = -pos[2]
        return np.array(list(orn) + list(lin_v) + list(ang_v) + [depth], dtype=np.float32)

    def _reward(self, obs, torque):
        rot = p.getMatrixFromQuaternion(obs[:4])
        body_fwd_axis = (-rot[1], -rot[4], -rot[7])        # -Y
        fwd_speed = np.dot(obs[4:7], body_fwd_axis)
        return fwd_speed - 0.01*torque**2

    def close(self):
        if p.isConnected():
            p.disconnect()

In [ ]:
p.resetDebugVisualizerCamera(
    cameraDistance=0.5,   # smaller = closer
    cameraYaw=50,
    cameraPitch=-20,
    cameraTargetPosition=[0, 0, -0.5])

error: Not connected to physics server.

In [ ]:
# inside FishEnv.reset(), replace the load line:
self.fish_id = p.loadURDF("simple_fish.urdf",
                          [0, 0, -0.5],
                          globalScaling=20)   # 20× bigger

error: Not connected to physics server.

In [31]:
env = FishEnv(render_mode="human")
obs, _ = env.reset()
for _ in range(2000):
    obs, _, _, _, _ = env.step(env.action_space.sample())
env.close()

error: Not connected to physics server.